In [97]:
import math
from math import pi
import matplotlib.pyplot as plt
import random
import itertools
from shapely.geometry import Polygon, Point

class Circle:
    def __init__(self, radius=1, n_points=10000):
        self.circle = [(math.cos(2*pi/n_points*point)*radius,math.sin(2*pi/n_points*point)*radius) for point in range(0,n_points+1)]    

    def draw_circle(self):
        y = [i[1] for i in c.circle]
        x = [i[0] for i in c.circle]
        plt.scatter(x, y)
        plt.scatter(0,0)

    def draw_chosen_points(self):
        y_points = [i[1] for i in self.points]
        x_points = [i[0] for i in self.points]
        plt.scatter(x_points, y_points)

    def draw_lines_between_points(self):
        y_points = [i[1] for i in self.points]
        x_points = [i[0] for i in self.points]
        x_line_points = x_points
        x_line_points.append(x_line_points[0])
        y_line_points = y_points
        y_line_points.append(y_line_points[0])
        plt.plot(x_line_points, y_line_points)
    
    def draw(self):
        self.draw_circle()
        self.draw_chosen_points()
        self.draw_lines_between_points()
        plt.show()
    
    def get_random_points_from_circle(self, n_points=5):
        points = list()
        while len(points) < n_points:
            point = random.choice(self.circle)
            if point not in points:
                points.append(point)
        self.points = points
        return points

    def check_if_point_is_contained_in_chosen_points(self, point=(0,0)):
        polygon = Polygon(self.points)
        point_of_interest = Point(point)
        return polygon.contains(point_of_interest)

n_runs = 100000
for run in range(3):
    count = 0
    c = Circle()
    for i in range(n_runs):
        points = c.get_random_points_from_circle()
        #c.draw()
        result = c.check_if_point_is_contained_in_chosen_points()
        if result:
            count+= 1

    print('Probability of origo lying inside triangle is {}'.format(round(count/n_runs, 3)))
    



Probability of origo lying inside triangle is 0.331
Probability of origo lying inside triangle is 0.332
Probability of origo lying inside triangle is 0.333
Probability of origo lying inside triangle is 0.331
Probability of origo lying inside triangle is 0.335
Probability of origo lying inside triangle is 0.333
Probability of origo lying inside triangle is 0.333
Probability of origo lying inside triangle is 0.331
Probability of origo lying inside triangle is 0.336


KeyboardInterrupt: 